# Evaluation
Calculating metrics for the evaluation of the trained models

In [314]:
# imports
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import numpy as np
from scipy.special import kl_div

In [315]:
# variables
data_input_filepath = './pickles/reservations_testing.pickle'

lstm_models_input_path = './pickles/results/demand/lstm_demand_prediction.pickle'
lstm_weather_models_input_path = './pickles/results/demand/lstm_weather_demand_prediction.pickle'
lstm_cc_models_input_path = './pickles/results/demand/lstm_cc_demand_prediction.pickle'
lstm_cc_weather_models_input_path = './pickles/results/demand/lstm_cc_weather_demand_prediction.pickle'
lstm_voronoi_cc_models_input_path = './pickles/results/demand/lstm_voronoi_cc_demand_prediction.pickle'
lstm_voronoi_cc_weather_models_input_path = './pickles/results/demand/lstm_voronoi_cc_weather_demand_prediction.pickle'

evaluation_output_path = './pickles/results/dl_evaluation_results.pickle'

In [316]:
# load data
validation_data = pd.read_pickle(data_input_filepath)

lstm_model_demand = pd.read_pickle(lstm_models_input_path)
lstm_weather_model_demand = pd.read_pickle(lstm_weather_models_input_path)
lstm_cc_model_demand = pd.read_pickle(lstm_cc_models_input_path)
lstm_cc_weather_model_demand = pd.read_pickle(lstm_cc_weather_models_input_path)
lstm_voronoi_cc_model_demand = pd.read_pickle(lstm_voronoi_cc_models_input_path)
lstm_voronoi_cc_weather_model_demand = pd.read_pickle(lstm_voronoi_cc_weather_models_input_path)

In [317]:
validation_data.set_index('startTime', inplace=True)
validation_data.sort_index(inplace=True)
validation_data.head()

,endTime,startLat,startLon,endLat,endLon,voronoi_grid_id,temperature,precipitation,small_grid_id,large_grid_id
startTime,,,,,,,,,,
2017-10-14 14:45:03,2017-10-14 15:05:06,48.777477,9.179767,48.772057,9.177928,592,20.4,0.0,3442,142
2017-10-14 14:58:06,2017-10-14 15:15:42,48.783566,9.157743,48.797287,9.208840,585,20.4,0.0,2558,98
2017-10-14 15:00:42,2017-10-14 15:06:43,48.767025,9.175741,48.767883,9.176202,514,20.0,0.0,3289,126
2017-10-14 16:09:02,2017-10-14 16:15:25,48.764015,9.167205,48.763809,9.176042,685,18.6,0.0,2917,111
2017-10-14 17:16:20,2017-10-14 17:41:23,48.761551,9.161797,48.772068,9.124672,696,17.6,0.0,2694,111


In [318]:
# create validation data to evaluate each model
lstm_validation_data = validation_data.resample('H').large_grid_id.value_counts().unstack().fillna(0)
lstm_validation_data.columns = lstm_validation_data.columns.map(str)
lstm_validation_data.sort_index(axis=1, inplace=True)

lstm_weather_validation_data = validation_data.resample('H').large_grid_id.value_counts().unstack().fillna(0)
lstm_weather_validation_data.columns = lstm_weather_validation_data.columns.map(str)
lstm_weather_validation_data.sort_index(axis=1, inplace=True)

lstm_cc_validation_data = validation_data.resample('H').small_grid_id.value_counts().unstack().fillna(0)
lstm_cc_validation_data.columns = lstm_cc_validation_data.columns.map(str)
lstm_cc_validation_data.sort_index(axis=1, inplace=True)

lstm_cc_weather_validation_data = validation_data.resample('H').small_grid_id.value_counts().unstack().fillna(0)
lstm_cc_weather_validation_data.columns = lstm_cc_weather_validation_data.columns.map(str)
lstm_cc_weather_validation_data.sort_index(axis=1, inplace=True)

lstm_voronoi_cc_validation_data = validation_data.resample('H').voronoi_grid_id.value_counts().unstack().fillna(0)
lstm_voronoi_cc_validation_data.columns = lstm_voronoi_cc_validation_data.columns.map(str)
lstm_voronoi_cc_validation_data.sort_index(axis=1, inplace=True)

lstm_voronoi_cc_weather_validation_data = validation_data.resample('H').voronoi_grid_id.value_counts().unstack().fillna(0)
lstm_voronoi_cc_weather_validation_data.columns = lstm_voronoi_cc_weather_validation_data.columns.map(str)
lstm_voronoi_cc_weather_validation_data.sort_index(axis=1, inplace=True)

In [319]:
# convert column names to strings
lstm_model_demand.columns = lstm_model_demand.columns.map(str)
lstm_weather_model_demand.columns = lstm_weather_model_demand.columns.map(str)
lstm_cc_model_demand.columns = lstm_cc_model_demand.columns.map(str)
lstm_cc_weather_model_demand.columns = lstm_cc_weather_model_demand.columns.map(str)
lstm_voronoi_cc_model_demand.columns = lstm_voronoi_cc_model_demand.columns.map(str)
lstm_voronoi_cc_weather_model_demand.columns = lstm_voronoi_cc_weather_model_demand.columns.map(str)

In [320]:
# add missing columns to both dataframes
def align_columns(df1, df2):
    combined = pd.concat([df1, df2], axis=1)

    # Reindex both dataframes to include all columns
    df1_aligned = df1.reindex(columns=combined.columns, fill_value=0)
    df2_aligned = df2.reindex(columns=combined.columns, fill_value=0)
    
    # sort columns alphabetically
    df1_aligned = df1_aligned[sorted(df1_aligned.columns)]
    df2_aligned = df2_aligned[sorted(df2_aligned.columns)]
    
    # sort values
    df1_aligned.sort_index(axis=1, inplace=True)
    df2_aligned.sort_index(axis=1, inplace=True)

    return df1_aligned, df2_aligned

## Calculate metrics

In [321]:
def normalize_array_columns(arr):
    # Normalize each column so that it sums to 1, avoiding division by zero
    column_sums = arr.sum(axis=0, keepdims=True)
    column_sums[column_sums == 0] = 1  
    return arr / column_sums

In [322]:
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    kl_1 = kl_div(normalize_array_columns(y_true), normalize_array_columns(y_pred))
    kl_2 = kl_div(normalize_array_columns(y_pred), normalize_array_columns(y_true))
    return [mse, mae, r2, np.median(kl_1), np.median(kl_2)]

In [323]:
# LSTM
lstm_validation_data, lstm_model_demand = align_columns(lstm_validation_data, lstm_model_demand)
lstm_model_demand_metrics = calculate_metrics(lstm_validation_data.values, lstm_model_demand.values)

In [324]:
# LSTM_weather
lstm_weather_validation_data, lstm_weather_model_demand = align_columns(lstm_weather_validation_data, lstm_weather_model_demand)
lstm_weather_model_demand_metrics = calculate_metrics(lstm_weather_validation_data.values, lstm_weather_model_demand.values)

In [325]:
# LSTM_CC
lstm_cc_validation_data, lstm_cc_model_demand = align_columns(lstm_cc_validation_data, lstm_cc_model_demand)
lstm_cc_model_demand_metrics = calculate_metrics(lstm_cc_validation_data.values, lstm_cc_model_demand.values)

In [326]:
# LSTM_CC_weather
lstm_cc_weather_validation_data, lstm_cc_weather_model_demand = align_columns(lstm_cc_weather_validation_data, lstm_cc_weather_model_demand)
lstm_cc_weather_model_demand_metrics = calculate_metrics(lstm_cc_weather_validation_data.values, lstm_cc_weather_model_demand.values)

In [327]:
# LSTM_Voronoi_CC
lstm_voronoi_cc_validation_data, lstm_voronoi_cc_model_demand = align_columns(lstm_voronoi_cc_validation_data, lstm_voronoi_cc_model_demand)
lstm_voronoi_cc_model_demand_metrics = calculate_metrics(lstm_voronoi_cc_validation_data.values, lstm_voronoi_cc_model_demand.values)

In [328]:
# LSTM_Voronoi_CC_weather
lstm_voronoi_cc_weather_validation_data, lstm_voronoi_cc_weather_model_demand = align_columns(lstm_voronoi_cc_weather_validation_data, lstm_voronoi_cc_weather_model_demand)
lstm_voronoi_cc_weather_model_demand_metrics = calculate_metrics(lstm_voronoi_cc_weather_validation_data.values, lstm_voronoi_cc_weather_model_demand.values)

In [329]:
# create dataframe with metrics
model_evaluation_metrics = pd.DataFrame({'model': ['lstm', 'lstm_weather', 'lstm_cc', 'lstm_cc_weather', 'lstm_voronoi_cc', 'lstm_voronoi_cc_weather'], 'mse': [lstm_model_demand_metrics[0], lstm_weather_model_demand_metrics[0], lstm_cc_model_demand_metrics[0], lstm_cc_weather_model_demand_metrics[0], lstm_voronoi_cc_model_demand_metrics[0], lstm_voronoi_cc_weather_model_demand_metrics[0]], 'mae': [lstm_model_demand_metrics[1], lstm_weather_model_demand_metrics[1], lstm_cc_model_demand_metrics[1], lstm_cc_weather_model_demand_metrics[1], lstm_voronoi_cc_model_demand_metrics[1], lstm_voronoi_cc_weather_model_demand_metrics[1]], 'r2': [lstm_model_demand_metrics[2], lstm_weather_model_demand_metrics[2], lstm_cc_model_demand_metrics[2], lstm_cc_weather_model_demand_metrics[2], lstm_voronoi_cc_model_demand_metrics[2], lstm_voronoi_cc_weather_model_demand_metrics[2]], 'kl_1': [lstm_model_demand_metrics[3], lstm_weather_model_demand_metrics[3], lstm_cc_model_demand_metrics[3], lstm_cc_weather_model_demand_metrics[3], lstm_voronoi_cc_model_demand_metrics[3], lstm_voronoi_cc_weather_model_demand_metrics[3]], 'kl_2': [lstm_model_demand_metrics[4], lstm_weather_model_demand_metrics[4], lstm_cc_model_demand_metrics[4], lstm_cc_weather_model_demand_metrics[4], lstm_voronoi_cc_model_demand_metrics[4], lstm_voronoi_cc_weather_model_demand_metrics[4]]})
model_evaluation_metrics

,model,mse,mae,r2,kl_1,kl_2
0,lstm,0.028057,0.059605,-10.580745,0.000072,inf
1,lstm_weather,0.033568,0.064695,-24.384932,0.000071,inf
2,lstm_cc,0.000951,0.002973,-0.043849,0.000071,inf
3,lstm_cc_weather,0.001042,0.005058,-0.168338,0.000070,inf
4,lstm_voronoi_cc,0.000722,0.002496,-0.000699,0.000027,inf
5,lstm_voronoi_cc_weather,0.000790,0.004218,-0.000699,0.000018,inf


In [330]:
# save metrics
model_evaluation_metrics.to_pickle(evaluation_output_path)